[View in Colaboratory](https://colab.research.google.com/github/pasumarthi/DeepLearning/blob/master/CIFAR10_exp_32F.ipynb)

In [1]:
#@title Default title text
#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,SGD

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
import os
print( os.getcwd() )
print( os.listdir() )

/content
['datalab', '.cache', 'CIFAR10_Batch419-0.606.hdf5', 'CIFAR10_Batch421-0.611.hdf5', '.rnd', 'CIFAR10_Batch425-0.638.hdf5', '.config', 'CIFAR10_Batch403-0.485.hdf5', '.local', '.nv', 'CIFAR10_Batch409-0.564.hdf5', '.ipython', 'CIFAR10_Batch415-0.581.hdf5', '.keras', '.forever']


In [0]:
batch_size = 64
num_classes = 10
epochs = 50
layers = 20
num_filter = 32
compression = 0.6
dropout_rate = 0.3
name='CIFAR10_Batch4'
data_augmentation = False

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#print(x_train.shape[1:])

32 32 3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2, layers=6):
    global compression
    print(layers)
    temp = input
    for _ in range(layers):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = MaxPooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [23]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
First_Conv2D = Conv2D(num_filter*2, (3,3),use_bias=False )(input)
Second_Con2D =Conv2D(num_filter*4,(7,1),use_bias=False)(First_Conv2D)
Third_Con2D=Conv2D(num_filter, (1,7), use_bias = False)(Second_Con2D)
Forth_Con2D=Conv2D(num_filter, (1,1), use_bias = False)(Third_Con2D)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate,layers=3)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate,layers=9)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate,layers=9)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate=12)
output = output_layer(Last_Block)

3
9
9
6


In [25]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_74 (Conv2D)              (None, 30, 30, 64)   1728        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, 30, 30, 64)   256         conv2d_74[0][0]                  
__________________________________________________________________________________________________
activation_60 (Activation)      (None, 30, 30, 64)   0           batch_normalization_60[0][0]     
__________________________________________________________________________________________________
conv2d_78 

In [0]:
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

In [0]:
lr_schedule = [10, 20, 30] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.1
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.02 # lr_decay_ratio = 0.2
    elif (epoch_idx + 1) < lr_schedule[2]:
        return 0.004
    return 0.0008

In [0]:
import keras.callbacks as callbacks
from keras.callbacks import LearningRateScheduler
save_c = callbacks.ModelCheckpoint(name + '{epoch:02d}-{val_acc:.3f}.hdf5', monitor="val_acc", save_best_only=True,save_weights_only=True,mode='max',period=2)
lrs = LearningRateScheduler(schedule=schedule)
callbacks_list = [save_c]

In [29]:
from keras.preprocessing.image import ImageDataGenerator
data_augmentation=True
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
    

Using real-time data augmentation.


In [0]:
#train_datagen=datagen.fit(x_train, augment=True)
#validate_datagen=datagen.fit(x_test, augment=True)
# compute quantities required for featurewise normalization
#datagen.fit(x_train, augment=True)
#datagen.fit(x_test, augment=True)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [30]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size= 128),
          callbacks=[save_c],
          epochs=45,
          verbose=1,
          validation_data=(x_test, y_test)
          )

Epoch 1/45
373/391 [===========================>..] - ETA: 6s - loss: 1.7065 - acc: 0.3790

391/391 [==============================] - 160s 409ms/step - loss: 1.6940 - acc: 0.3833 - val_loss: 3.1636 - val_acc: 0.2991
Epoch 2/45
391/391 [==============================] - 148s 380ms/step - loss: 1.3077 - acc: 0.5246 - val_loss: 3.9369 - val_acc: 0.2759
Epoch 3/45
 37/391 [=>............................] - ETA: 2:09 - loss: 1.2407 - acc: 0.5608

391/391 [==============================] - 149s 380ms/step - loss: 1.1493 - acc: 0.5844 - val_loss: 2.5054 - val_acc: 0.4129
Epoch 4/45
328/391 [========================>.....] - ETA: 22s - loss: 1.0516 - acc: 0.6230

391/391 [==============================] - 148s 377ms/step - loss: 1.0465 - acc: 0.6256 - val_loss: 1.9024 - val_acc: 0.4844
Epoch 5/45
391/391 [==============================] - 147s 377ms/step - loss: 0.9748 - acc: 0.6527 - val_loss: 2.2882 - val_acc: 0.4451
Epoch 6/45
 27/391 [=>............................] - ETA: 2:10 - loss: 0.9388 - acc: 0.6612

391/391 [==============================] - 148s 377ms/step - loss: 0.9090 - acc: 0.6774 - val_loss: 2.6403 - val_acc: 0.4762
Epoch 7/45
325/391 [=======================>......] - ETA: 23s - loss: 0.8635 - acc: 0.6935

...

391/391 [==============================] - 149s 381ms/step - loss: 0.4083 - acc: 0.8576 - val_loss: 0.7097 - val_acc: 0.8135
Epoch 40/45
322/391 [=======================>......] - ETA: 24s - loss: 0.4088 - acc: 0.8590

391/391 [==============================] - 149s 382ms/step - loss: 0.4089 - acc: 0.8591 - val_loss: 1.0397 - val_acc: 0.7727
Epoch 41/45
391/391 [==============================] - 149s 381ms/step - loss: 0.4014 - acc: 0.8614 - val_loss: 0.6293 - val_acc: 0.8428
Epoch 42/45
 25/391 [>.............................] - ETA: 2:11 - loss: 0.3876 - acc: 0.8656

391/391 [==============================] - 149s 382ms/step - loss: 0.3975 - acc: 0.8628 - val_loss: 0.8270 - val_acc: 0.8041
Epoch 43/45
322/391 [=======================>......] - ETA: 24s - loss: 0.3940 - acc: 0.8617

391/391 [==============================] - 150s 383ms/step - loss: 0.3937 - acc: 0.8617 - val_loss: 1.0967 - val_acc: 0.7566
Epoch 44/45
391/391 [==============================] - 150s 384ms/step - loss: 0.3945 - acc: 0.8635 - val_loss: 0.6852 - val_acc: 0.8203
Epoch 45/45
 25/391 [>.............................] - ETA: 2:10 - loss: 0.3571 - acc: 0.8681

391/391 [==============================] - 150s 384ms/step - loss: 0.3831 - acc: 0.8668 - val_loss: 1.3919 - val_acc: 0.7200


In [31]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 1.3918771629333495
Test accuracy: 0.72


In [32]:
!ls

CIFAR10_Batch402-0.276.hdf5  CIFAR10_Batch420-0.732.hdf5
CIFAR10_Batch403-0.485.hdf5  CIFAR10_Batch421-0.611.hdf5
CIFAR10_Batch404-0.484.hdf5  CIFAR10_Batch424-0.735.hdf5
CIFAR10_Batch408-0.570.hdf5  CIFAR10_Batch425-0.638.hdf5
CIFAR10_Batch409-0.564.hdf5  CIFAR10_Batch426-0.751.hdf5
CIFAR10_Batch412-0.644.hdf5  CIFAR10_Batch428-0.779.hdf5
CIFAR10_Batch414-0.692.hdf5  CIFAR10_Batch430-0.797.hdf5
CIFAR10_Batch415-0.581.hdf5  CIFAR10_Batch436-0.836.hdf5
CIFAR10_Batch419-0.606.hdf5  datalab


In [33]:
!dir

CIFAR10_Batch402-0.276.hdf5  CIFAR10_Batch420-0.732.hdf5
CIFAR10_Batch403-0.485.hdf5  CIFAR10_Batch421-0.611.hdf5
CIFAR10_Batch404-0.484.hdf5  CIFAR10_Batch424-0.735.hdf5
CIFAR10_Batch408-0.570.hdf5  CIFAR10_Batch425-0.638.hdf5
CIFAR10_Batch409-0.564.hdf5  CIFAR10_Batch426-0.751.hdf5
CIFAR10_Batch412-0.644.hdf5  CIFAR10_Batch428-0.779.hdf5
CIFAR10_Batch414-0.692.hdf5  CIFAR10_Batch430-0.797.hdf5
CIFAR10_Batch415-0.581.hdf5  CIFAR10_Batch436-0.836.hdf5
CIFAR10_Batch419-0.606.hdf5  datalab


In [34]:
# Save the trained weights in to .h5 format
model.save_weights("CIFAR_exp_32F.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('CIFAR_exp_32F.h5')

In [0]:
# Saving using PyDrive to google drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'DNST_model40_45.h5'})
uploaded.SetContentFile('DNST_model40_45.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
#downloaded = drive.CreateFile({'id': uploaded.get('id')})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 17WSS58RK1BlM_OdTMTK98G_Nq38t5tHl


In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)